In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

#from models_all import *
from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 2000
N_bound = 100

la = np.array([1,1])
lb = np.array([0,0])

#x,y
X_train = lb+(la-lb)*lhs(2,N_train)
x_inside = X_train[:,0:1]
y_inside = X_train[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
y_inside = numpy_to_tensor(y_inside, var_name="y_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

data_bound_x0 = np.hstack((np.zeros((N_bound, 1)), lb[1] + (la[1] - lb[1]) * lhs(1, N_bound)))
data_bound_x1 = np.hstack((np.ones((N_bound, 1)), lb[1] + (la[1] - lb[1]) * lhs(1, N_bound)))

data_bound_y0 = np.hstack((lb[0] + (la[0] - lb[0]) * lhs(1, N_bound), np.zeros((N_bound, 1))))
data_bound_y1 = np.hstack((lb[0] + (la[0] - lb[0]) * lhs(1, N_bound), np.ones((N_bound, 1))))

data_bound = np.vstack((data_bound_x0, data_bound_x1, data_bound_y0, data_bound_y1))
data_bound = numpy_to_tensor(data_bound, var_name="data_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [3]:
def exact_u(x, y):
    return (1 / (2 * np.pi**2)) * np.sin(np.pi * x) * np.sin(np.pi * y)

In [ ]:
plot_data = np.concatenate((np.linspace(0,1,200).reshape(-1,1),np.linspace(0,1,200).reshape(-1,1)),1)

plot_data_x = plot_data[:,0:1]
plot_data_y = plot_data[:,1:2]

plot_data_X,plot_data_Y = np.meshgrid(plot_data_x,plot_data_y)
plot_data_numpy = np.concatenate((plot_data_X.reshape(-1,1),plot_data_Y.reshape(-1,1)),1)

plot_data_tensor = torch.from_numpy(plot_data_numpy).float()
plot_data_tensor = plot_data_tensor.cuda()

aa = exact_u(plot_data_numpy[:,0],plot_data_numpy[:,1])
plt.imshow(aa.reshape(200,200), interpolation='nearest',extent=[0, 1, 0, 1], cmap='rainbow')
plt.colorbar(shrink=.98)

plt.show()

In [ ]:
np.random.seed(5678)
n_test_data = 1000

test_data = lb+(la-lb)*lhs(2,n_test_data)
test_u = exact_u(test_data[:,0:1], test_data[:,1:2])

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

## Loss_f

In [6]:
def get_loss_f(x_grad, y_grad, PINNs, return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs1(torch.cat((x_grad,y_grad),1))
    
    E_x = compute_higher_order_derivatives(E_inside, [x_grad])
    E_xx = compute_higher_order_derivatives(E_x, [x_grad])  
    E_y = compute_higher_order_derivatives(E_inside, [y_grad])
    E_yy = compute_higher_order_derivatives(E_y, [y_grad])
    
    deata_E = E_xx+E_yy
    
    loss_term = deata_E+torch.sin(torch.tensor(np.pi)*x_grad)*torch.sin(torch.tensor(np.pi)*y_grad)
    ########### loss f  ###########
    
    if return_sequence=='yes':
        return torch.square(loss_term)
    else:
        return torch.mean(torch.square(loss_term))

## loss_bound

In [7]:
def get_loss_bound(bound_x, bound_y, PINNs, return_sequence='not'):

    E_bound_x_0 = PINNs1(torch.cat((bound_x,torch.zeros_like(bound_x)),1))
    E_bound_x_1 = PINNs1(torch.cat((bound_x,torch.ones_like(bound_x)),1))
    
    E_bound_y_0 = PINNs1(torch.cat((torch.zeros_like(bound_y),bound_y),1))
    E_bound_y_1 = PINNs1(torch.cat((torch.ones_like(bound_y),bound_y),1))
    
    loss_bound_for_bound_x_0 = torch.mean(torch.square(E_bound_x_0))
    loss_bound_for_bound_x_1 = torch.mean(torch.square(E_bound_x_1))
    
    loss_bound_for_bound_y_0 = torch.mean(torch.square(E_bound_y_0))
    loss_bound_for_bound_y_1 = torch.mean(torch.square(E_bound_y_1))
    
    loss_bound_value = loss_bound_for_bound_x_0+loss_bound_for_bound_x_1+loss_bound_for_bound_y_0+loss_bound_for_bound_y_1
    
    return loss_bound_value

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA on GPU")
else:
    device = "cpu"
    print("Using CPU")

# PINN

In [9]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [10]:
class hidden_layers(nn.Module):
    def __init__(self,input_number,output_number):
        super(hidden_layers, self).__init__()
        self.layer = nn.Linear(input_number,output_number)
    def forward(self, x):
        x = self.layer(x)
        x = torch.tanh(x)
        return x

class NN_H2 (nn.Module):
    def __init__(self,in_N, width, depth, out_N):
        super(NN_H2, self).__init__()
        self.in_N = in_N
        self.width = width
        self.depth = depth
        self.out_N = out_N

        self.stack = nn.ModuleList()

        self.stack.append(hidden_layers(in_N, width))

        for i in range(depth):
            self.stack.append(hidden_layers(width, width))

        self.stack.append(nn.Linear(width, out_N))
        
        
    def forward(self, x):
        for m in self.stack:
            x = m(x)
        return x

def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        #nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

In [11]:
PINNs1 = NN_H2(2, 20, 4, 1)
PINNs1.apply(weights_init)
PINNs1.cuda()

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [12]:
##############PW_L_f###############
e_k = 1e-4
q_k = 0.01
s_k = 0.3
Ada_CL_L_f = Ada_CL(e_k,q_k,s_k,device)
weight_sequence_L_f = (1/N_train)*np.ones((N_train,1))
##############PW_L_f###############


########### record list ###########
loss_all_1 = []
loss_f_1 = []
loss_b_1 = []
test_loss_1 = []
plot_abs_loss = []
beta_number_list = []
########### record list ###########

In [ ]:
nIter1 = 2000
it = 0

while  it<2000:
    
    ##### loss bound  ######
    E_bound = PINNs1(data_bound)
    loss_bound = torch.mean(torch.square(E_bound))
    
    ##############PW_L_f###############
    loss_f = get_loss_f(x_inside,y_inside,PINNs1,return_sequence='not')
    ##############PW_L_f###############
    
    #########loss PI#########
    loss = loss_f+3*loss_bound
    #########loss PI#########
    
    #########  test_loss NRMSE  #########
    pre_u = PINNs1(test_data)  
    test_loss = relative_l2_torch(pre_u,test_u)
    test_loss_1.append(test_loss)
    #########  test_loss NRMSE  #########
    
    ##############Record###############
    loss_all_1.append(loss.item())
    loss_f_1.append(loss_f.item())
    loss_b_1.append(loss_bound.item())
    
    loss_f_sequence = get_loss_f(x_inside,y_inside,PINNs1,return_sequence='yes').cpu().detach().numpy()
    beta_number_value = np.ones_like(loss_f_sequence)
    beta_number_value = np.sum(beta_number_value[loss_f_sequence>e_k])/N_train
    beta_number_list.append(beta_number_value)              
    ##############Record Original###############
    
    
    if it%20 == 0 and it<2000:
        plot_abs_loss.append(np.abs(aa-PINNs1(plot_data_tensor).cpu().detach().numpy().reshape(-1,)))
    
    if it % 500 == 0:
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
    
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    it = it + 1        
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)